# Overview

This Jupyter Notebook takes in data from a Google Sheet that contains stop change details and their associated high level categories and outputs a JSON file for each line to be used in the MyBus tool.

The output file is used by the MyBus tool's results page and contains the Stop-level changes that are displayed there.

Run all cells to output files in the format `#-changes.json`

As of 5/28/21, this should output data for 141 lines.

In [57]:
import pandas as pd
import numpy as np
from zipfile import ZipFile

DATA_INPUT_PATH = '../data/input/'
DATA_OUTPUT_PATH = '../data/stop-changes/'
SCRATCH_PATH = 'scratch/'
LINES_PATH = '../data/lines.json'
GOOGLE_SHEET_LINK = "https://docs.google.com/spreadsheets/d/e/2PACX-1vQC-r5V2B0xw886UtXO5TU1YXPy_HzyBoQXQOjDWD_qwmBBlvn3fyVsa-BHu0FymPM8gk4O_UxMtJsK/pub?output=csv"

## Load GTFS Data

Use `lines.json`, which was generated from the `line-dropdown` notebook.

Extract `stops.zip` to the scratch folder.

In [58]:
lines = pd.read_json(LINES_PATH)
lines_array = lines.loc[:, 'route_number'].values
lines.head()

,route_id,route_short_name,route_number
0,2,2,2
1,4,4,4
2,10,10,10
3,14,14,14
4,16,16,16


In [59]:
with ZipFile('../data/input/stops.zip', 'r') as zf:
    zf.extractall(SCRATCH_PATH)

STOPS_PATH = SCRATCH_PATH + 'stops.txt'

In [60]:
stops = pd.read_csv(STOPS_PATH,
    usecols=['stop_id','stop_name'],
    dtype={'stop_id':'int','stop_name':'string'})

stops.head()

,stop_id,stop_name
0,1,Paramount / Slauson
1,3,Jefferson / 10th
2,6,120th / Augustus F Hawkins
3,7,120th / Martin Luther King Hospital
4,12,15054 Sherman Way


## Load Stop Changes Data

Data was compiled from spreadsheets and slides provided by Service Planners. Pull the data directly from the Google Sheet.

In [61]:
# stop_changes = pd.read_csv(DATA_INPUT_PATH + 'stop_changes - ALL.csv',
#     usecols={'line', 'stop_id', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled'})
stop_changes = pd.read_csv(GOOGLE_SHEET_LINK, usecols={'line', 'direction', 'stop_id', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled', 'replaced_by_micro', 'service_restored'})

stop_changes.head()

,line,direction,stop_id,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled,replaced_by_micro,service_restored
0,2,Westbound,14805,False,False,False,True,False,False,False,False,False
1,2,Westbound,16318,False,False,False,False,True,False,False,False,False
2,2,Eastbound,6424,False,False,False,True,False,False,False,False,False
3,4,Westbound,16318,False,False,False,False,True,False,False,False,False
4,4,Eastbound,5972,False,False,False,True,False,False,False,False,False


In [62]:
# merge with stops.txt to get stop names
#stop_changes.stop_id = stop_changes.stop_id.astype('int')
stop_changes_with_names = pd.merge(stop_changes, stops, how='inner', on='stop_id')
stop_changes_with_names = stop_changes_with_names[['line', 'stop_id', 'stop_name', 'direction', 'service_canceled', 'service_changed', 'service_replaced', 'stop_canceled', 'stop_relocated', 'route_changed', 'owl_service_canceled', 'replaced_by_micro', 'service_restored']]
stop_changes_with_names.to_csv(SCRATCH_PATH + 'stop_changes_with_names.csv')

stop_changes_with_names.head()

,line,stop_id,stop_name,direction,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled,replaced_by_micro,service_restored
0,2,14805,Sunset / Mapleton,Westbound,False,False,False,True,False,False,False,False,False
1,2,16318,Sunset / Alvarado,Westbound,False,False,False,False,True,False,False,False,False
2,4,16318,Sunset / Alvarado,Westbound,False,False,False,False,True,False,False,False,False
3,704,16318,Sunset / Alvarado,Westbound,False,False,True,False,False,False,False,False,False
4,2,6424,Sunset / Mapleton,Eastbound,False,False,False,True,False,False,False,False,False


## Analyze Data

### Stops in `stop_changes` not in GTFS

Outputs results to file `stops-not-in-gtfs.json` in the `scratch` folder.

In [63]:
stops_not_in_gtfs = stop_changes[~stop_changes.stop_id.isin(stops.stop_id)]
stops_not_in_gtfs = stops_not_in_gtfs[['line', 'stop_id']]
stops_not_in_gtfs = stops_not_in_gtfs.sort_values(['line', 'stop_id'])

stops_not_in_gtfs.to_json(SCRATCH_PATH + 'stops-not-in-gtfs.json', orient='records')

stops_not_in_gtfs.head(20)

,line,stop_id
16,4,14381
17,4,14409
34,18,1269
56,152,4580
80,169,12107
85,656,3600
249,690,9031
248,690,10572


### Duplicate `line` & `stop_id` combos

Outputs results to file `stops-changes-duplicates.json` in the `scratch` folder.

In [64]:
# Find duplicate line + stop_id combos

# exclude non-existent stop_ids
stop_changes_existing_stopids = stop_changes.loc[stop_changes['stop_id'] != 0]
stop_changes_existing_stopids = stop_changes_existing_stopids.groupby(['line', 'stop_id', 'direction']).size().reset_index(name="count")
stop_changes_existing_stopids = stop_changes_existing_stopids.loc[stop_changes_existing_stopids['count'] > 1]

# Output file with the stops that have duplicate rows.
filtered_combos = pd.merge(stop_changes_existing_stopids, stop_changes, how='inner', on=['line', 'stop_id', 'direction']).sort_values(by=['line', 'stop_id', 'direction'])
filtered_combos.to_csv(SCRATCH_PATH + 'stop-changes-duplicates.csv')

filtered_combos.head(20)

,count,line,direction,stop_id,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled,replaced_by_micro,service_restored


## Add Stops for Restored Service

Lines 110 and 550 have restored service.  The stops do not exist in the June 2021 GTFS data.  Reference GitHub [issue #62](https://github.com/LACMTA/mybus/issues/62).

In [65]:
restored_stops = pd.read_csv('../data/input/restored_stops.csv',
	usecols={'line', 'stop_id','direction'},
	dtype={'line':'int','stop_id':'string','direction':'string'})

restored_stops['service_canceled'] = False
restored_stops['service_changed'] = False
restored_stops['service_replaced'] = False
restored_stops['stop_canceled'] = False
restored_stops['stop_relocated'] = False
restored_stops['route_changed'] = False
restored_stops['owl_service_canceled'] = False
restored_stops['replaced_by_micro'] = False
restored_stops['service_restored'] = True

In [66]:
stop_changes

,line,direction,stop_id,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled,replaced_by_micro,service_restored
0,2,Westbound,14805,False,False,False,True,False,False,False,False,False
1,2,Westbound,16318,False,False,False,False,True,False,False,False,False
2,2,Eastbound,6424,False,False,False,True,False,False,False,False,False
3,4,Westbound,16318,False,False,False,False,True,False,False,False,False
4,4,Eastbound,5972,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
396,733,Westbound,4096,False,False,True,False,False,False,False,False,False
397,733,Westbound,29,False,False,True,False,False,False,False,False,False
398,854,Eastbound,1172,False,False,False,True,False,True,False,False,False
399,854,Eastbound,1177,False,False,False,True,False,True,False,False,False


In [67]:
stop_changes = pd.concat([stop_changes,restored_stops])
stop_changes

,line,direction,stop_id,service_canceled,service_changed,service_replaced,stop_canceled,stop_relocated,route_changed,owl_service_canceled,replaced_by_micro,service_restored
0,2,Westbound,14805,False,False,False,True,False,False,False,False,False
1,2,Westbound,16318,False,False,False,False,True,False,False,False,False
2,2,Eastbound,6424,False,False,False,True,False,False,False,False,False
3,4,Westbound,16318,False,False,False,False,True,False,False,False,False
4,4,Eastbound,5972,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
41,550,Southbound,15616,False,False,False,False,False,False,False,False,True
42,550,Southbound,10994,False,False,False,False,False,False,False,False,True
43,550,Southbound,10853,False,False,False,False,False,False,False,False,True
44,550,Southbound,2324,False,False,False,False,False,False,False,False,True


## Output Stop Changes Data

Iterate through lines.json and output a file for each line.

In [68]:
# Loop through line numbers and output a file for each line with all data for that line
line_count = 1
stop_count = 0

for line in lines_array:
    subset = stop_changes[stop_changes.line == line]
    filename = DATA_OUTPUT_PATH + str(line) + '-changes.json'
    
    stop_count += len(subset)
    print(subset[['line', 'direction', 'stop_id']])
    subset.to_json(filename, orient='records')
    
    print('********** ' + 'Line ' + filename + ' created (' + str(len(subset)) + ')')
    print('********** ' + str(stop_count)  + ' total stops')
    print('********** ' + str(line_count)  + ' total lines')
    line_count += 1

   line  direction stop_id
0     2  Westbound   14805
1     2  Westbound   16318
2     2  Eastbound    6424
********** Line ../data/stop-changes/2-changes.json created (3)
********** 3 total stops
********** 1 total lines
    line  direction stop_id
3      4  Westbound   16318
4      4  Eastbound    5972
5      4  Eastbound    1070
6      4  Eastbound    5978
7      4  Eastbound    5971
8      4  Eastbound    6064
9      4  Eastbound    5983
10     4  Eastbound    5987
11     4  Eastbound    6054
12     4  Eastbound    5984
13     4  Eastbound    6043
14     4  Eastbound    5994
15     4  Eastbound    6046
16     4  Westbound   14381
17     4  Westbound   14409
18     4  Westbound   14427
19     4  Westbound   14374
20     4  Westbound   14425
21     4  Westbound   14365
22     4  Westbound   14434
23     4  Westbound   14366
24     4  Westbound   14364
25     4  Westbound   14444
26     4  Westbound   14351
27     4  Westbound   14358
28     4  Westbound    1091
29     4  Westbound   